# Trie des données

### Introduction:

Ce notebook à pour but d'effectuer un premier tri des données pour répondre à notre problématique, dans le cadre du projet "infrastructure de recharge" 

### Jeu de données:

je me suis focaliser sur la région Ile de France, sur les 3 dernières années disponibles: 2019 à 2021 , les années antérieures à 2019 ne sont pas pertinentes, au vu de l'évolution du prix de l'immobilier.

Les données ont été préalablement importés, fusionnées dans le notebook nommé valeur foncière_1 et sauvegardés dans un fichier csv nommé IDF_2019_to_2021.csv

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
pd.pandas.set_option('display.max_columns',None)

In [48]:
# Importer depuis le local 
df= pd.read_csv(r"C:\Users\alaro\Downloads\Dataset\IDF_2019_to_2021.csv")
df.head()

C:\Users\alaro\AppData\Local\Temp\ipykernel_2348\817029412.py:2: DtypeWarning: Columns (14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(r"C:\Users\alaro\Downloads\Dataset\IDF_2019_to_2021.csv")


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2019-1499564,2019-01-04,1,Vente,1196000.0,17.0,NaN,RUE DUPHOT,2999,75001.0,75101,Paris 1er Arrondissement,75,NaN,NaN,75101000BC0014,NaN,NaN,21,106.85,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,112.0,3.0,NaN,NaN,NaN,NaN,NaN,2.325288,48.868416
1,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,182,NaN,50.0,102.8,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,104.0,3.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305
2,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305
3,2019-1499566,2019-01-08,1,Vente,100000.0,1.0,NaN,RUE MAGELLAN,5903,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000AP0016,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.300028,48.869577
4,2019-1499567,2019-01-03,1,Vente,5400.0,79.0,NaN,RUE DES GRAVILLIERS,4302,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AV0022,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,21.0,1.0,NaN,NaN,NaN,NaN,NaN,2.353479,48.864674


A la lecture de la documentation fournie par le site de Gouvimmo, et après une première audite des données brutes, nous pouvons faire les constatations suivantes:
- Chaque mutations (ventes) est identifiée par un numéro unique.
- Une même mutation peut être constituée de plusieurs lignes. Chaque ligne décrit un bien vendu dans la même mutation.
- Les différents types de biens sont répartit dans trois colonnes de type object, à plusieurs modalités. 
- Une première colonne décrit les biens construits, et comporte 4 modalités: Maisons, appartement, dépendance et local industriel ou commercial.
- Deux autres colonnes décrivent le type de terrain. On y trouve notamment la mention de terrain à bâtir. 
- La valeur foncière (valeur de la vente) n'est pas détaillée pour chaque ligne. Seule le montant totale est renseigné, et est répété dans chaque lignes.
- Un certain nombre de mutations comportent un très grand nombre de lignes, avec plusieurs appartements ou maisons, et de nombreux terrains annexes et dépendances


Au regard de ces constats nous avons opté pour la stratégie suivante:
- Ne conserver que les mutations  comportant  terrain à batir 
- Ne pas conserver les mutations ne correspondant pas à des ventes.

Ces différents critères nous permettrons de ne conserver que des mutations concerné par la vente d'un terrain à batir.

# Préparation et premier nettoyage du dataframe:

### Préparation:

Dans le but d'effectuer ce tri, nous allons commencer par regrouper tous les types de terrains autre que "terraine à bâtir" en une seule modalité "Autre terrain"

Cette opération nous permettra par la suite de compter le nombre de chaque type de biens pour chaque mutations, et ainsi faire le trie.

In [49]:
#Transformation des types de terrains autre que terrains à bâtir en "autre terrain"
df.loc[(df["nature_culture"]!="terrains a bâtir") & (df["nature_culture"].isna()==False), "nature_culture"]="Autre terrain"
df["nature_culture"].value_counts(dropna=False)
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2019-1499564,2019-01-04,1,Vente,1196000.0,17.0,NaN,RUE DUPHOT,2999,75001.0,75101,Paris 1er Arrondissement,75,NaN,NaN,75101000BC0014,NaN,NaN,21,106.85,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,112.0,3.0,NaN,NaN,NaN,NaN,NaN,2.325288,48.868416
1,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,182,NaN,50.0,102.8,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,104.0,3.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305
2,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305
3,2019-1499566,2019-01-08,1,Vente,100000.0,1.0,NaN,RUE MAGELLAN,5903,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000AP0016,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.300028,48.869577
4,2019-1499567,2019-01-03,1,Vente,5400.0,79.0,NaN,RUE DES GRAVILLIERS,4302,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AV0022,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,21.0,1.0,NaN,NaN,NaN,NaN,NaN,2.353479,48.864674


Dans de nombreux cas, une même ligne contiens un bien construit (maison ou appartement), mais également un terrain à bâtir (qui à servi de support à la construction). Elles peuvent également comporter un terrain autre (Un jardin par exemple). Pour que ces lignes soient bien considérées comme des biens construits et non comme des terrains vides, nous allons créer les variables "terrain à bâtir" et "autre terrain" en ne considérant que les lignes ne comportant pas de maison ou appartement. 

In [50]:
#Dichotomisation des types de locaux
df = df.join(pd.get_dummies(df['type_local']))
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison
0,2019-1499564,2019-01-04,1,Vente,1196000.0,17.0,NaN,RUE DUPHOT,2999,75001.0,75101,Paris 1er Arrondissement,75,NaN,NaN,75101000BC0014,NaN,NaN,21,106.85,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,112.0,3.0,NaN,NaN,NaN,NaN,NaN,2.325288,48.868416,1,0,0,0
1,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,182,NaN,50.0,102.8,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,104.0,3.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305,1,0,0,0
2,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305,0,1,0,0
3,2019-1499566,2019-01-08,1,Vente,100000.0,1.0,NaN,RUE MAGELLAN,5903,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000AP0016,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.300028,48.869577,0,1,0,0
4,2019-1499567,2019-01-03,1,Vente,5400.0,79.0,NaN,RUE DES GRAVILLIERS,4302,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AV0022,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,21.0,1.0,NaN,NaN,NaN,NaN,NaN,2.353479,48.864674,1,0,0,0


In [51]:
#Création de colonnes terrains_a_batir (= ligne terrain à bâtir ne contenant pas d'appartement ou de maison)
df.loc[(df["nature_culture"]=="terrains a bâtir") & ((df["Appartement"]+df["Maison"]+df["Dépendance"]==0)), "terrains_a_batir"]=1

#Création de colonnes terrains_autre (= ligne autre terrain ne contenant pas d'appartement ou de maison)
df.loc[(df["nature_culture"]=="Autre terrain") & ((df["Appartement"]+df["Maison"]+df["Dépendance"]==0)), "terrains_autre"]=1

df[["terrains_a_batir", "terrains_autre"]] = df[["terrains_a_batir", "terrains_autre"]].fillna(0)

display(df.head())
display(df["terrains_a_batir"].value_counts(dropna=False))
display(df["terrains_autre"].value_counts(dropna=False))

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison,terrains_a_batir,terrains_autre
0,2019-1499564,2019-01-04,1,Vente,1196000.0,17.0,NaN,RUE DUPHOT,2999,75001.0,75101,Paris 1er Arrondissement,75,NaN,NaN,75101000BC0014,NaN,NaN,21,106.85,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,112.0,3.0,NaN,NaN,NaN,NaN,NaN,2.325288,48.868416,1,0,0,0,0.0,0.0
1,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,182,NaN,50.0,102.8,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,104.0,3.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305,1,0,0,0,0.0,0.0
2,2019-1499565,2019-01-03,1,Vente,1570490.0,13.0,NaN,RUE DE THORIGNY,9298,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AL0015,NaN,NaN,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.363076,48.860305,0,1,0,0,0.0,0.0
3,2019-1499566,2019-01-08,1,Vente,100000.0,1.0,NaN,RUE MAGELLAN,5903,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000AP0016,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.300028,48.869577,0,1,0,0,0.0,0.0
4,2019-1499567,2019-01-03,1,Vente,5400.0,79.0,NaN,RUE DES GRAVILLIERS,4302,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AV0022,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,21.0,1.0,NaN,NaN,NaN,NaN,NaN,2.353479,48.864674,1,0,0,0,0.0,0.0


0.0    1082506
1.0      21426
Name: terrains_a_batir, dtype: int64

0.0    977931
1.0    126001
Name: terrains_autre, dtype: int64

### Nettoyage: Lignes non associée à terrain à batir 

Nous allons vérifier si toutes les lignes de notre dataframe sont associées à terrain à batir ou autres terrain ou si certaines lignes ne sont rattachées à aucun de ces types 

In [52]:
#Nombre de mutations comportant des lignes ne correspondant pas à un bien principale ou secondaire ou un terrain à batir:
df[(df["terrains_a_batir"]+df["terrains_autre"])==0].shape[0]

956505

Nous constatons qu'il y a des mutations comportant des lignes ne correspondant pas  à un terrain à batir. Il s'agit probablement de mutations comportant des biens de type industriel ou commercial, ou à des mutations pour lesquelles certaines lignes n'ont pas été correctement complétés à la vente. Ces mutations risquent d'êtres difficilement interprétables et doivent être retirées.

Pour supprimer totalement les mutations consernées, nous allons donc récupérer la liste de leurs ID de mutation, et filtrer le dataframe avec cette liste.

In [53]:
#Récupération de la liste des mutations consernées:
toremove=df[(df["terrains_a_batir"]+df["terrains_autre"])==0]["id_mutation"].unique()

In [54]:
#Retrait des mutations consernées dans le dataframe:
df=df.loc[df["id_mutation"].isin(toremove)==False]

In [55]:
#Vérification après nettoyage:
df[(df["terrains_a_batir"]+df["terrains_autre"])==0].shape[0]

0

In [56]:
#Retrait des mutations autre que terrain à batir et autres_terrain:
toremove2=df[(df["Appartement"]+df["Maison"]+df["Dépendance"]>0)]["id_mutation"].unique()
df=df.loc[df["id_mutation"].isin(toremove2)==False]
df[(df["Appartement"]+df["Maison"]+df["Dépendance"]>0)].shape[0]

0

In [57]:
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison,terrains_a_batir,terrains_autre
228,2019-1499715,2019-01-25,2,Vente,51500000.0,106.0,NaN,RUE DU TEMPLE,9191,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AR0025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,Autre terrain,NaN,NaN,1383.0,2.358118,48.862446,0,0,0,0,0.0,1.0
344,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,464.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
345,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,100.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
346,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,17.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
347,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,464.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0


### Nettoyage: Valeur foncières par mutations:

Selon la documentation de Gouvimmo, nous nous attendons à ce que la valeur foncière représente le prix de vente totale et soit identique pour chaque lignes d'une même mutation. Nous allons vérifier que cette règle est bien respecté.

In [58]:
#Vérification que la valeur foncière est systématiquement la même indiqué pour toute les lignes d'une même mutation:
valeur_fonciere=df[["id_mutation", "valeur_fonciere"]].groupby('id_mutation' , as_index = True).agg({"valeur_fonciere": ["mean", "min", "max"]})

print("Nombre de mutations avec des valeurs foncières différentes renseignées pour une même mutation: ", 
      sum(valeur_fonciere["valeur_fonciere"]["mean"]!=valeur_fonciere["valeur_fonciere"]["max"]))

Nombre de mutations avec des valeurs foncières différentes renseignées pour une même mutation:  377


Nous constatons que cette règle n'est pas respecté pour certaines mutations. Nous devons les retirer de notre jeu de données. Nous allons donc récupérer la liste des ID de mutations consernées pour filtrer notre dataframe.

In [59]:
#Récupération de la liste des mutations consernées:
toremove=valeur_fonciere[(valeur_fonciere["valeur_fonciere"]["mean"]!=valeur_fonciere["valeur_fonciere"]["max"])==True].index

#Suppression des mutations consernées:
df=df.loc[df["id_mutation"].isin(toremove)==False]

#Vérification après nettoyage:
valeur_fonciere=df[["id_mutation", "valeur_fonciere"]].groupby('id_mutation' , as_index = True).agg({"valeur_fonciere": ["mean", "min", "max"]})

print("Nombre de mutations avec des valeurs foncières différentes renseignées pour une même mutation: ", 
      sum(valeur_fonciere["valeur_fonciere"]["mean"]!=valeur_fonciere["valeur_fonciere"]["max"]))

Nombre de mutations avec des valeurs foncières différentes renseignées pour une même mutation:  0


### Nettoyage: Etude des NaN:

In [60]:
pd.DataFrame(df.isnull().sum() * 100 / len(df), columns = ["% missing"]).sort_values(by="% missing")

,% missing
id_mutation,0.000000
Maison,0.000000
Local industriel. commercial ou assimilé,0.000000
Dépendance,0.000000
Appartement,0.000000
surface_terrain,0.000000
nature_culture,0.000000
code_nature_culture,0.000000
nombre_lots,0.000000
terrains_a_batir,0.000000


Nous constatons que certaines colonnes ne sont pas pertinentes pour notre dataframe:
- La colonne numero_volume est totalement vide et peut être retiré.
- Les colonnes ancien_code_commune, ancien_nom_commune, ancien_id_parcelle, nature_culture_speciale et code_nature_culture_speciale sont peu utiles et très largements vides. Elles peuvent être supprimées.
 Nous pouvons donc nous passer de ces colonnes, ainsi que des colonnes numero_lot.

In [61]:
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison,terrains_a_batir,terrains_autre
228,2019-1499715,2019-01-25,2,Vente,51500000.0,106.0,NaN,RUE DU TEMPLE,9191,75003.0,75103,Paris 3e Arrondissement,75,NaN,NaN,75103000AR0025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,Autre terrain,NaN,NaN,1383.0,2.358118,48.862446,0,0,0,0,0.0,1.0
344,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,464.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
345,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,100.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
346,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,17.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0
347,2019-1499785,2019-01-31,1,Vente,3400000.0,10.0,NaN,RUE DE STOCKHOLM,9098,75008.0,75108,Paris 8e Arrondissement,75,NaN,NaN,75108000BY0053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,Local industriel. commercial ou assimilé,464.0,0.0,S,Autre terrain,NaN,NaN,128.0,2.322082,48.877168,0,0,1,0,0.0,1.0


In [63]:
#Suppression des colonnes innutiles:
df=df.drop(['numero_volume',
            "ancien_code_commune",
            "ancien_nom_commune",
            "ancien_id_parcelle",
            "nature_culture_speciale",
            "code_nature_culture_speciale"], axis=1)

In [64]:
#Suppression des colonnes lotx_surface_carrez et lotx_numero:

df=df.drop(["lot1_surface_carrez",
            "lot2_surface_carrez",
            "lot3_surface_carrez",
            "lot4_surface_carrez",
            "lot5_surface_carrez", 
            "lot1_numero",
            "lot2_numero",
            "lot3_numero",
            "lot4_numero",
            "lot5_numero"], axis=1)

Pour finir, les colonnes code_type_local, type_local, code_nature_culture et nature_culture ont été dichotomisées et ne sont donc plus utiles car redondantes avec les colonnes créées par dichotomisation. Elles peuvent donc être retirées.

In [65]:
#Suppression des colonnes code_type_local, type_local, code_nature_culture et nature_culture:

df=df.drop(["code_type_local", "type_local", "code_nature_culture", "nature_culture"], axis=1)

# Creation du dataframe concaténant les données pour chaque mutations:

Afin de pouvoir étudier correctement le prix de vente en fonction de toute les caractéristiques de chaque mutations, nous souhaitons regrouper les données afin de n'avoir plus qu'une seule ligne par mutation. Nous allons appliquer la stratégie suivante:
- Les lignes seront regroupés par id de mutation par la méthode groupby.
- La valeur foncière totale étant recopiée sur chaque ligne d'une même mutation, celle ci sera regroupé en utilisant "mean" avec la méthode groupby.
- Le nombre_lots, surface_terrain et les données dichotomisées seront regroupés en utilisant "sum" avec la méthode groupby.
- Les données relatives à date_mutation, numero_disposition, nature_mutation, addresse complète, N° de parcelle, longitude et latitude seront conservées uniquement pour le  terrain à bâtir,

Pour ces dernières données (données en type object), nous allons donc commencer par les supprimer des lignes ne correspondant pas à terrain à batir. Nous pourrons ensuite récupérer les données des biens principaux en utilisant "first" avec la méthode groupby.

In [66]:
#Suppression des données relatives à date_mutation, numero_disposition, nature_mutation, addresse complète, 
#N° de parcelle, longitude et latitude pour les lignes a un terrain à bâtir:

df.loc[( df["terrains_a_batir"])==0, ["date_mutation",
                                        "numero_disposition", 
                                        "nature_mutation", 
                                        "adresse_numero",
                                        "adresse_suffixe",
                                        "adresse_nom_voie",
                                        "adresse_code_voie",
                                        "code_postal",
                                        "code_commune",
                                        "nom_commune",
                                        "code_departement",
                                        "id_parcelle",
                                        "longitude",
                                        "latitude"]]=None

Nous pouvons maintenant regrouper les mutations avec la méthode groupby sur l'ID de mutation.

In [67]:
#Création du dataframe groupé par ID de mutation:
df=df.groupby('id_mutation' , as_index = False).agg({"valeur_fonciere": "mean",
                                                    "date_mutation": "first",
                                                    "numero_disposition": "first",
                                                    "nature_mutation": "first",
                                                    "adresse_numero": "first",
                                                    "adresse_suffixe": "first",
                                                    "adresse_nom_voie": "first",
                                                    "adresse_code_voie": "first",
                                                    "code_postal": "first",
                                                    "code_commune": "first",
                                                    "nom_commune": "first",
                                                    "code_departement": "first",
                                                    "id_parcelle": "first",
                                                    "nombre_lots": "sum",
                                                    "surface_terrain": "sum",
                                                    "longitude": "first",
                                                    "latitude": "first",
                                                    "Local industriel. commercial ou assimilé": "sum",
                                                    "terrains_a_batir": "sum",
                                                    "terrains_autre": "sum",
                                                    "date_mutation": "first"})


In [68]:
df.head()

,id_mutation,valeur_fonciere,date_mutation,numero_disposition,nature_mutation,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,nombre_lots,surface_terrain,longitude,latitude,Local industriel. commercial ou assimilé,terrains_a_batir,terrains_autre
0,2019-1135503,165000.0,2019-01-03,1.0,Vente,NaN,None,RUE DES BORDES,0100,77310.0,77407.0,Saint-Fargeau-Ponthierry,77.0,77407000BC0428,0,489.0,2.534703,48.531012,0,2.0,0.0
1,2019-1135512,257000.0,2019-01-10,1.0,Vente en l'état futur d'achèvement,NaN,None,LA SORTIE DE GUIGNES,B025,77390.0,77222.0,Guignes,77.0,77222000AH0297,0,199.0,2.806298,48.628694,0,1.0,0.0
2,2019-1135554,382000.0,None,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,None,0,433.0,NaN,NaN,1,0.0,1.0
3,2019-1135555,635000.0,None,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,None,0,2501.0,NaN,NaN,1,0.0,2.0
4,2019-1135564,1.0,None,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,None,0,74.0,NaN,NaN,0,0.0,1.0


# Application des filtres:

Dans cette partie, nous applicons les filtres au dataframe selon la stratégie définie plus haut: 
- Suppression des mutations comportant des locaux industriels ou commercial
- Suppression des mutations ne comportant pas de terrain à bâtir 
- Suppression des mutations ne correspondant pas à des ventes

In [70]:
#Suppression des terrains autre non associées à un terrain à bâtir 
df=df.drop(axis = 0, index=df[(df["terrains_autre"]>0) & ((df["terrains_a_batir"])==0)].index)

Après avoir appliqué notre stratégie de filtrage, nous nous attendons à ce que les mutations de notre dataframe comportent que des terrain à batir et terrain-autre attaché a ce dernier

In [76]:
#Vérification des carractéristiques de notre df:
df.iloc[:, 13:].describe()

,nombre_lots,surface_terrain,longitude,latitude,Local industriel. commercial ou assimilé,terrains_a_batir,terrains_autre
count,11124.000000,11124.000000,10888.000000,10888.000000,11124.0,11124.000000,11124.000000
mean,0.001259,1201.419274,2.419839,48.796110,0.0,1.542700,0.317781
std,0.035455,4514.173546,0.367464,0.199657,0.0,2.983408,1.608428
min,0.000000,1.000000,1.476424,48.138730,0.0,1.000000,0.000000
25%,0.000000,287.000000,2.175715,48.634304,0.0,1.000000,0.000000
50%,0.000000,424.000000,2.418611,48.821739,0.0,1.000000,0.000000
75%,0.000000,750.250000,2.682770,48.952272,0.0,1.000000,0.000000
max,1.000000,147194.000000,3.515560,49.217550,0.0,152.000000,82.000000


Nous nous occupons maintenant des natures de mutation. Nous affichons les différentes modalités de la variable:

In [77]:
#Affichage des modalités des natures de mutations:
df["nature_mutation"].value_counts()

Vente                                 8567
Vente en l'état futur d'achèvement    1891
Vente terrain à bâtir                  603
Echange                                 53
Adjudication                             6
Expropriation                            4
Name: nature_mutation, dtype: int64

# Pour ne conserver que les mutations de nature vente, nous supprimons celles correspondants à des échanges, expropriations ou adjudication, Vente en l'état futur d'achèvement

In [78]:
#Suppression des mutations correspondant à des échanges, expropriations ou adjudication:
df=df[(df["nature_mutation"]!="Echange") & (df["nature_mutation"]!="Expropriation") & (df["nature_mutation"]!="Adjudication")& (df["nature_mutation"]!="Vente en l'état futur d'achèvement")]

#Vérification après nettoyage:
df["nature_mutation"].value_counts()

Vente                    8567
Vente terrain à bâtir     603
Name: nature_mutation, dtype: int64

Nous faisons maintenant une dernière vérification des NaN restants:

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9170 entries, 0 to 32106
Data columns (total 21 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id_mutation                               9170 non-null   object 
 1   valeur_fonciere                           9170 non-null   float64
 2   date_mutation                             9170 non-null   object 
 3   numero_disposition                        9170 non-null   float64
 4   nature_mutation                           9170 non-null   object 
 5   adresse_numero                            5096 non-null   float64
 6   adresse_suffixe                           926 non-null    object 
 7   adresse_nom_voie                          9170 non-null   object 
 8   adresse_code_voie                         9170 non-null   object 
 9   code_postal                               9168 non-null   float64
 10  code_commune                       

Il reste des NaN dans les colonnes adresse_numero, adresse_nom_voie et adresse_suffixe. Ces NaN peuvent être des valeurs manquantes qui seraient auquel cas non récupérables, soit des valeurs inexistantes dans la réalité. Ces NaN seront donc laissé comme tel.

Il reste 6 NaN dans la colonne code_postal. Ceux ci peuvent être facilement récupérés via le nom de comune et la voie, ou le code commune.

In [80]:
#Affichage des NaN dans la colonne code_postal:
df[df["code_postal"].isna()==True][["code_postal", "code_commune", "nom_commune", "adresse_nom_voie"]]

,code_postal,code_commune,nom_commune,adresse_nom_voie
15367,NaN,78650.0,Le Vésinet,PL JOSEPHINE BAKER
15370,NaN,78650.0,Le Vésinet,PL JOSEPHINE BAKER


Le code postale de Le Vésinet est le 78110;

In [81]:
#Remplacement des codes postaux NaN:

df.loc[(df["code_postal"].isna()==True) & (df["nom_commune"]=="Le Vésinet"), "code_postal"]=78110


In [82]:
#Réinitialisation de l'index pour le dataframe final:
df=df.reset_index(drop=True)
df.head()

,id_mutation,valeur_fonciere,date_mutation,numero_disposition,nature_mutation,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,nombre_lots,surface_terrain,longitude,latitude,Local industriel. commercial ou assimilé,terrains_a_batir,terrains_autre
0,2019-1135503,165000.0,2019-01-03,1.0,Vente,NaN,None,RUE DES BORDES,0100,77310.0,77407.0,Saint-Fargeau-Ponthierry,77.0,77407000BC0428,0,489.0,2.534703,48.531012,0,2.0,0.0
1,2019-1135568,110000.0,2019-01-08,1.0,Vente terrain à bâtir,NaN,None,L ORME BRISE,B016,77310.0,77378.0,Pringy,77.0,77378000AD0220,0,184.0,2.564561,48.521392,0,1.0,0.0
2,2019-1135616,183000.0,2019-01-09,1.0,Vente,54.0,None,AV VICTOR HUGO,0646,77170.0,77053.0,Brie-Comte-Robert,77.0,77053000AV0166,0,367.0,2.608611,48.682525,0,1.0,0.0
3,2019-1135650,187000.0,2019-01-21,1.0,Vente,NaN,None,AV LAFAYETTE,0580,77680.0,77390.0,Roissy-en-Brie,77.0,773900000D2584,0,445.0,2.660324,48.791354,0,1.0,0.0
4,2019-1135698,92000.0,2019-01-21,1.0,Vente terrain à bâtir,NaN,None,MACHAULT,B044,77133.0,77266.0,Machault,77.0,772660000F0966,0,351.0,2.828720,48.454006,0,1.0,0.0


In [83]:
df.describe()

,valeur_fonciere,numero_disposition,adresse_numero,code_postal,code_commune,code_departement,nombre_lots,surface_terrain,longitude,latitude,Local industriel. commercial ou assimilé,terrains_a_batir,terrains_autre
count,9.170000e+03,9170.000000,5096.000000,9170.000000,9170.000000,9170.000000,9170.000000,9170.000000,8951.000000,8951.000000,9170.0,9170.000000,9170.000000
mean,6.207157e+05,1.058233,92.242347,84768.388659,84631.170883,84.354635,0.001527,1352.736205,2.434037,48.785770,0.0,1.556598,0.358342
std,5.955132e+06,0.278840,685.668090,7725.364133,7729.335121,7.739380,0.039046,4917.582018,0.377826,0.199195,0.0,3.080112,1.742581
min,1.000000e+00,1.000000,1.000000,75012.000000,75112.000000,75.000000,0.000000,1.000000,1.476424,48.138730,0.0,1.000000,0.000000
25%,1.090000e+05,1.000000,7.000000,77470.000000,77336.000000,77.000000,0.000000,325.000000,2.181873,48.629423,0.0,1.000000,0.000000
50%,1.450000e+05,1.000000,17.000000,78720.500000,78513.000000,78.000000,0.000000,480.000000,2.429143,48.812565,0.0,1.000000,0.000000
75%,2.100000e+05,1.000000,38.000000,91800.000000,91667.000000,91.000000,0.000000,846.750000,2.711155,48.945793,0.0,1.000000,0.000000
max,2.892000e+08,8.000000,9103.000000,95870.000000,95680.000000,95.000000,1.000000,147194.000000,3.515560,49.217550,0.0,152.000000,82.000000


# Export du dataframe en csv

Nous exportons maintenant notre dataframe finale en un fichier csv que nous nommons IDF_2019_to_2021_v2.csv

Celui ci servira de base pour la partie data viz.

In [85]:
df.to_csv(r"C:\Users\alaro\Downloads\Dataset\IDF_2019_to_2021_v2.csv", index=False)